# Entrenamiento

## 1. Preprocesado

### 1.1. Lectura

In [19]:
import json
url, names, all_dictionaries = "./../generated/ml/models/", [ 'dictionary_manual_values.json' ], {}

for name in names:
    destiny = url + name.replace(' ', '%20')
    try:
        f = open(destiny, 'rb')
        all_dictionaries=json.loads(f.read())
    except OSError:
        print("No hay diccionario previo")
        all_dictionaries={}

In [20]:
url, names, all_dictionaries = "./../generated/ml/models/", [ 'dictionary_manual_values.json' ], {}

for name in names:
    destiny = url + name.replace(' ', '%20')
    try:
        f = open(destiny, 'rb')
        all_dictionaries=json.loads(f.read())
    except OSError:
        print("No hay diccionario previo")
        all_dictionaries={}

In [21]:
all_dictionaries

{'make': {'ALFA ROMEO': 500,
  'AUDI': 1500,
  'MAZDA': 2500,
  'BMW': 3500,
  'CITROEN': 4500,
  'DACIA': 5500,
  'FIAT': 6500,
  'FORD': 7500,
  'HONDA': 8500,
  'HYUNDAI': 9500,
  'JAGUAR': 10500,
  'JEEP': 11500,
  'KIA': 12500,
  'LEXUS': 13500,
  'MERCEDES-BENZ': 14500,
  'MITSUBISHI': 15500,
  'MINI': 16500,
  'NISSAN': 17500,
  'PORSCHE': 18500,
  'OPEL': 19500,
  'SKODA': 20500,
  'PEUGEOT': 21500,
  'VOLVO': 22500,
  'RENAULT': 23500,
  'SEAT': 24500,
  'TOYOTA': 25500,
  'VOLKSWAGEN': 26500},
 'transmissionType': {'COMMON.TRANSMISSION_AUTOMATIC': 0,
  'COMMON.TRANSMISSION_MANUAL': 1},
 'seller_type': {'profesional': 0},
 'bodyType': {'BODY_TYPE_4X4': 500,
  'BODY_TYPE_SEDAN': 700,
  'BODY_TYPE_CABRIO': 900,
  'BODY_TYPE_FAMILIAR': 1100,
  'BODY_TYPE_COUPE': 1300,
  'BODY_TYPE_MINIVAN': 1500}}

In [22]:
import pandas as pd

url, names, dataframes = "./../generated/ml/preprocess/", [ 'all.csv' ], []

for name in names:
  destiny = url + name.replace(' ', '%20')
  print('Cargando: ' + destiny)
  if 'csv' in name:
    dataframes.append(pd.read_csv(destiny, delimiter=',', decimal=".", thousands=None))
  else:
    dataframes.append(pd.read_json(destiny))

Cargando: ./../generated/ml/preprocess/all.csv


In [23]:
all = pd.concat(dataframes)

### 1.2. Procesado de datos nulos

In [24]:
print("Datos iniciales | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.dropna(axis=1,how='all')
print("Eliminados na or null on columns | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.dropna(axis=0)
print("Eliminados na or null on rows | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
all=all.drop_duplicates()
all=all.drop_duplicates(subset=['id'], keep='first')
print("Eliminados duplicados | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))
print("Datos finales | Longitud: {} | Num Columns: {}".format(len(all), len(all.columns)))

Datos iniciales | Longitud: 5086 | Num Columns: 36
Eliminados na or null on columns | Longitud: 5086 | Num Columns: 36
Eliminados na or null on rows | Longitud: 5086 | Num Columns: 36
Eliminados duplicados | Longitud: 5086 | Num Columns: 36
Datos finales | Longitud: 5086 | Num Columns: 36


### 1.3. Procesado columnas

In [25]:
columns_to_drop=['id','color','fuelTypeId','makeId','province','provinceId','title','transmissionTypeId','brand_id','luggageCapacity','consumptionAverage','consumptionExtraUrban','model_id','manufacturerPrice','seats','weight','emissions','fuelTankCapacity','height','luggageCapacity','brand','doors','topSpeed','consumptionUrban','model']
all=all.drop(columns_to_drop, axis = 1)

In [26]:
def columns_numeric_type(df,columns_names):
  for columns_name in columns_names:
    df[column_name]=pd.to_numeric(df[column_name])
  return df

In [27]:
qualitative_variable_names = [ e for e in all.select_dtypes(['object']).columns ]
for column_name in qualitative_variable_names:
  print(column_name, len(all[column_name].unique()))

make 27
transmissionType 2
seller_type 1
bodyType 6


In [28]:
import operator

def replace_for_numeric(df,column_name,initial,increaser,value_dictionary={}):
  values=df[column_name].unique()
  pos=initial
  if len(value_dictionary) > 0:
    max_dictionary=max(value_dictionary.items(), key=operator.itemgetter(1))[1]
  else:
    max_dictionary=initial
  if max_dictionary > pos:
    pos=max_dictionary+increaser
  for value in values:
    if value not in value_dictionary:
      value_dictionary[value]=pos
      df.loc[df[column_name] == value, column_name] = pos
      pos=pos+increaser
    else:
      df.loc[df[column_name] == value, column_name] = value_dictionary[value]
  df[column_name]=pd.to_numeric(df[column_name])
  # print("Nuevos valores: ", df[column_name].unique())
  return df,value_dictionary

In [29]:
all,word_dictionary=replace_for_numeric(all,'make',500,1000,all_dictionaries.get('make', {}))
all_dictionaries['make']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'transmissionType',0,1,all_dictionaries.get('transmissionType', {}))
all_dictionaries['transmissionType']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'seller_type',0,1,all_dictionaries.get('seller_type', {}))
all_dictionaries['seller_type']=word_dictionary
all,word_dictionary=replace_for_numeric(all,'bodyType',500,200,all_dictionaries.get('bodyType', {}))
all_dictionaries['bodyType']=word_dictionary
print(all_dictionaries)

{'make': {'ALFA ROMEO': 500, 'AUDI': 1500, 'MAZDA': 2500, 'BMW': 3500, 'CITROEN': 4500, 'DACIA': 5500, 'FIAT': 6500, 'FORD': 7500, 'HONDA': 8500, 'HYUNDAI': 9500, 'JAGUAR': 10500, 'JEEP': 11500, 'KIA': 12500, 'LEXUS': 13500, 'MERCEDES-BENZ': 14500, 'MITSUBISHI': 15500, 'MINI': 16500, 'NISSAN': 17500, 'PORSCHE': 18500, 'OPEL': 19500, 'SKODA': 20500, 'PEUGEOT': 21500, 'VOLVO': 22500, 'RENAULT': 23500, 'SEAT': 24500, 'TOYOTA': 25500, 'VOLKSWAGEN': 26500}, 'transmissionType': {'COMMON.TRANSMISSION_AUTOMATIC': 0, 'COMMON.TRANSMISSION_MANUAL': 1}, 'seller_type': {'profesional': 0}, 'bodyType': {'BODY_TYPE_4X4': 500, 'BODY_TYPE_SEDAN': 700, 'BODY_TYPE_CABRIO': 900, 'BODY_TYPE_FAMILIAR': 1100, 'BODY_TYPE_COUPE': 1300, 'BODY_TYPE_MINIVAN': 1500}}


In [30]:
all.tail(2)

,km,make,transmissionType,year,price,seller_type,bodyType,cubicCapacity,hp,acceleration,length,width
5084,64914,26500,0,2016,25990,0,700,1968.0,184.0,7.5,4268.0,1799.0
5085,66760,26500,0,2019,36099,0,500,1968.0,190.0,7.9,4486.0,1839.0


### 1.5. Split

In [31]:
from sklearn.model_selection import train_test_split as tts

all_x_data = all.drop('price', axis = 1)
all_y_data = all['price']

x_train, x_test, y_train, y_test = tts(all_x_data, all_y_data, random_state = 1, test_size = 0.20)

def print_bests(results, limit=5):
  best_results=sorted(results, key = lambda x: x.get('score'), reverse=True)[:limit]
  for best_result in best_results:
    print("score: {} | model_name: {}".format(best_result.get('score'), best_result.get('model_name')))
  return best_results

## 2. Entrenamientos

### 2.8. Entrenamiento MLPClassifier

In [32]:
from sklearn.neural_network import MLPRegressor as MLP

param_activation=['identity', 'logistic', 'tanh', 'relu']
param_solver=['lbfgs', 'sgd', 'adam']
param_max_iter=30000
num_columns=len(all.columns)*2
param_hidden_layer=range(num_columns, num_columns+1)
mlp_networks=[]

# for activation_parameter in param_activation:
#   for solver_parameter in param_solver:
#     for hidden_layer_parameter in param_hidden_layer:
#       mlp_networks.append(MLP(hidden_layer_sizes=hidden_layer_parameter, activation=activation_parameter, solver=solver_parameter, random_state=1, max_iter=param_max_iter))

mlp_networks.append(MLP(hidden_layer_sizes=num_columns, random_state=1, max_iter=param_max_iter))

mlp_networks

[MLPRegressor(hidden_layer_sizes=24, max_iter=30000, random_state=1)]

In [33]:
for network in mlp_networks:
  print(network.get_params(True))
  network.fit(x_train, y_train)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': 24, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 30000, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 1, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}


In [34]:
mlp_models=[]

for model in mlp_networks:
  score=network.score(x_test, y_test)
  mlp_models.append({'score': score, 'model': model, 'model_name': model.__class__.__name__})

## 3. Resultados

In [35]:
best_results=mlp_models

best_results=print_bests(best_results,1)

score: 0.7972161360831678 | model_name: MLPRegressor


## 4. Guardado de modelos

In [36]:
import pickle
import json

for i in range(len(best_results)):
  model_to_save = best_results[i]
  with open("./../generated/ml/models/neural/" + str(i) + "_" + model_to_save.get('model_name') + "_manual_values_9.pkl", "wb") as file:
    pickle.dump(model_to_save['model'], file)

with open("./../generated/ml/models/dictionary_manual_values.json", "w") as file:
  file.write(json.dumps(all_dictionaries))